In [1]:
import torch
import torch.nn as nn

In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}

### A Placeholder GPT model architecture class

In [3]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb = nn.Embedding(cfg['context_length'], cfg['emb_dim'])
        self.drop_emb = nn.Dropout(cfg['drop_rate'])
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg['n_layers'])]
        )
        self.final_norm = DummyLayerNorm(cfg['emb_dim'])
        self.out_head = nn.Linear(
            cfg['emb_dim'], cfg['vocab_size'], bias = False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device = in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        return x

class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps = 1e-5):
        super().__init__()

    def forward(self, x):
        return x

### Tokenizer using tiktoken

In [4]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))

In [5]:
print(batch)

[tensor([6109, 3626, 6100,  345]), tensor([6109, 1110, 6622,  257])]


In [6]:
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [7]:
batch.shape

torch.Size([2, 4])

In [8]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch) # here the batch is the stacked version of embeddings of txt1 and txt2

In [9]:
print("shape of the logits:\n", logits.shape)
print(logits)

shape of the logits:
 torch.Size([2, 4, 50257])
tensor([[[-1.2034,  0.3201, -0.7130,  ..., -1.5548, -0.2390, -0.4667],
         [-0.1192,  0.4539, -0.4432,  ...,  0.2392,  1.3469,  1.2430],
         [ 0.5307,  1.6720, -0.4695,  ...,  1.1966,  0.0111,  0.5835],
         [ 0.0139,  1.6755, -0.3388,  ...,  1.1586, -0.0435, -1.0400]],

        [[-1.0908,  0.1798, -0.9484,  ..., -1.6047,  0.2439, -0.4530],
         [-0.7860,  0.5581, -0.0610,  ...,  0.4835, -0.0077,  1.6621],
         [ 0.3567,  1.2698, -0.6398,  ..., -0.0162, -0.1296,  0.3717],
         [-0.2407, -0.7349, -0.5102,  ...,  2.0057, -0.3694,  0.1814]]],
       grad_fn=<UnsafeViewBackward0>)


In [10]:
print(logits.shape)

torch.Size([2, 4, 50257])


### Example for layer normalization

In [42]:
torch.manual_seed(123)
batch_example = torch.randn(2,5)

In [43]:
print(batch_example)

tensor([[-0.1115,  0.1204, -0.3696, -0.2404, -1.1969],
        [ 0.2093, -0.9724, -0.7550,  0.3239, -0.1085]])


In [44]:
layer = nn.Sequential(nn.Linear(5,6), nn.ReLU())
out = layer(batch_example)

In [45]:
print(out)

tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)


In [46]:
mean = out.mean(dim=1, keepdims=True)
variance = out.var(dim=1, keepdims=True)
print("Mean: ", mean)
print("variance: ", variance)

Mean:  tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)
variance:  tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>)


In [47]:
torch.set_printoptions(sci_mode=False)
out_norm = (out - mean) / torch.sqrt(variance)
mean = out_norm.mean(dim=1, keepdims=True)
variance = out_norm.var(dim=1, keepdims=True)
print("Mean: ", mean)
print("variance: ", variance)

Mean:  tensor([[    0.0000],
        [    0.0000]], grad_fn=<MeanBackward1>)
variance:  tensor([[1.],
        [1.]], grad_fn=<VarBackward0>)


In [48]:
print(out_norm)

tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)


### Implementing layer normalization

In [49]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdims=True)
        var = x.var(dim=-1, keepdims=True, unbiased = False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [50]:
model = LayerNorm(emb_dim=5)
out_ln = model(batch_example)
mean = out_ln.mean(dim=-1, keepdims=True)
var = out_ln.var(dim=-1, keepdims=True, unbiased = False)
print(mean, " ", var)

tensor([[    -0.0000],
        [     0.0000]], grad_fn=<MeanBackward1>)   tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)
